In [1]:
import pandas as pd
import datetime
import numpy as np

Pre-praparing umsatz 

In [2]:
dates_umsatz_null = pd.to_datetime([
    "2013-12-25", "2013-12-26", "2014-01-01",
    "2014-04-18", "2014-05-01", "2014-05-03",
    "2014-05-04", "2014-08-17", "2014-12-25",
    "2014-12-26", "2015-01-01", "2015-04-03",
    "2015-05-01", "2015-12-25", "2015-12-26",
    "2016-01-01", "2016-03-24", "2016-03-25",
    "2016-07-07", "2016-07-18", "2016-08-14",
    "2016-08-15", "2016-12-17", "2016-12-25",
    "2016-12-26", "2017-01-01", "2017-04-14",
    "2017-05-01", "2017-07-04", "2017-10-23",
    "2017-10-31", "2017-12-25", "2017-12-26",
    "2018-01-01", "2018-03-30", "2018-05-01",
    "2018-05-21"
])

#format dates_umsatz_null as dataframe
df_umsatz_null = pd.DataFrame(dates_umsatz_null, columns=['Datum']) 

print(df_umsatz_null)

        Datum
0  2013-12-25
1  2013-12-26
2  2014-01-01
3  2014-04-18
4  2014-05-01
5  2014-05-03
6  2014-05-04
7  2014-08-17
8  2014-12-25
9  2014-12-26
10 2015-01-01
11 2015-04-03
12 2015-05-01
13 2015-12-25
14 2015-12-26
15 2016-01-01
16 2016-03-24
17 2016-03-25
18 2016-07-07
19 2016-07-18
20 2016-08-14
21 2016-08-15
22 2016-12-17
23 2016-12-25
24 2016-12-26
25 2017-01-01
26 2017-04-14
27 2017-05-01
28 2017-07-04
29 2017-10-23
30 2017-10-31
31 2017-12-25
32 2017-12-26
33 2018-01-01
34 2018-03-30
35 2018-05-01
36 2018-05-21


In [3]:
#create new dataframe with all dates from 2018-08-01 to 2019-07-31 and dates_umsatz_null
all_dates = pd.date_range(start='2018-08-01', end='2019-07-31')
all_dates_df = pd.DataFrame(all_dates, columns=['Datum'])

print(all_dates_df)


         Datum
0   2018-08-01
1   2018-08-02
2   2018-08-03
3   2018-08-04
4   2018-08-05
..         ...
360 2019-07-27
361 2019-07-28
362 2019-07-29
363 2019-07-30
364 2019-07-31

[365 rows x 1 columns]


In [5]:
missing_dates = pd.merge(all_dates_df, df_umsatz_null, on='Datum', how='outer', indicator=False)
print(missing_dates)

         Datum
0   2013-12-25
1   2013-12-26
2   2014-01-01
3   2014-04-18
4   2014-05-01
..         ...
397 2019-07-27
398 2019-07-28
399 2019-07-29
400 2019-07-30
401 2019-07-31

[402 rows x 1 columns]


In [ ]:
"""#check for what month and days (no year) warengruppe 6 is avalable every year
mask = df_inputated['Warengruppe'] == 6
# ensure the 'Datum' values are datetime-like before using .dt (do not overwrite original column)
date_series = pd.to_datetime(df_inputated.loc[mask, 'Datum'], errors='coerce')
warengruppe6_dates = date_series.dt.strftime('%m-%d').dropna().unique()
print("Warengruppe 6 is available on the following month-day combinations:")
print(warengruppe6_dates)"""

In [6]:
warengruppe6_dates = [
    '10-24', '10-25', '10-26', '10-27', '10-29', '10-30', '10-31',
    '11-01', '11-02', '11-03', '11-04', '11-05', '11-06', '11-07',
    '11-08', '11-09', '11-10', '11-11', '11-12', '11-13', '11-14',
    '11-15', '11-16', '11-17', '11-18', '11-19', '11-20', '11-21',
    '11-22', '11-23', '11-24', '11-25', '11-26', '11-27', '11-28',
    '11-29', '11-30', '12-01', '12-02', '12-03', '12-04', '12-05',
    '12-06', '12-07', '12-08', '12-09', '12-10', '12-11', '12-12',
    '12-13', '12-14', '12-15', '12-16', '12-17', '12-18', '12-19',
    '12-20', '12-21', '12-22', '12-23', '12-24', '12-27', '12-29',
    '12-30', '01-02', '10-28', '12-28', '01-03', '01-04', '01-05',
    '01-06', '12-31'
]

In [14]:
def prep_dataset(df_daten, warengruppe6_dates, dates_umsatz_null):
    df = df_daten.copy()

    # 1) Ensure Datum is datetime
    df['Datum'] = pd.to_datetime(df['Datum'])

    # 2) Create base with all unique dates
    base_dates = df['Datum'].drop_duplicates().sort_values().reset_index(drop=True)
    base = pd.DataFrame({'Datum': base_dates})

    # 3) Determine if each date should have 5 or 6 groups (compare MM-DD)
    wg6 = set(warengruppe6_dates)
    base['mmdd'] = base['Datum'].dt.strftime('%m-%d')
    base['n_groups'] = np.where(base['mmdd'].isin(wg6), 6, 5)

    # 4) Expand each date into 1..5 or 1..6 Warengruppe
    base['Warengruppe'] = base['n_groups'].apply(lambda n: list(range(1, n + 1)))
    base = base.explode('Warengruppe').drop(columns=['mmdd', 'n_groups'])
    base['Warengruppe'] = base['Warengruppe'].astype(int)

    # 5) Bring original Umsatz if it exists in df_daten
    if 'Warengruppe' in df.columns:
        df_model = base.merge(
            df[['Datum', 'Warengruppe', 'Umsatz']],
            on=['Datum', 'Warengruppe'],
            how='left'
        )
    else:
        # If df_daten has no Warengruppe column, create empty Umsatz column
        df_model = base.copy()
        df_model['Umsatz'] = np.nan

    # 6) Set Umsatz = 0 for dates in dates_umsatz_null
    if dates_umsatz_null is not None and len(dates_umsatz_null) > 0:
        dates_null = pd.to_datetime(dates_umsatz_null)
        mask_null = df_model['Datum'].dt.normalize().isin(dates_null.normalize())
        df_model.loc[mask_null, 'Umsatz'] = 0

    # Optional: fill remaining NaNs with 0
    # df_model['Umsatz'] = df_model['Umsatz'].fillna(0)

    # 7) Create id = YYMMDD + Warengruppe
    df_model['id'] = df_model['Datum'].dt.strftime('%y%m%d') + df_model['Warengruppe'].astype(str)

    # Order columns nicely
    cols = ['id', 'Datum', 'Warengruppe', 'Umsatz']
    other_cols = [c for c in df_model.columns if c not in cols]
    df_model = df_model[cols + other_cols]

    return df_model


In [18]:
df_missing = prep_dataset(missing_dates, warengruppe6_dates, dates_umsatz_null)

In [16]:
df_daten = pd.read_csv('/workspaces/bakery_prediction/0_DataPreparation/Raw/umsatzdaten_gekuerzt.csv')
print(df_daten.head())

        id       Datum  Warengruppe      Umsatz
0  1307011  2013-07-01            1  148.828353
1  1307021  2013-07-02            1  159.793757
2  1307031  2013-07-03            1  111.885594
3  1307041  2013-07-04            1  168.864941
4  1307051  2013-07-05            1  171.280754


In [29]:
# ensure Datum column in df_daten is datetime to match df_missing
df_daten['Datum'] = pd.to_datetime(df_daten['Datum'])

# Ensure 'id' dtypes match before merging (avoid int vs object error).
# Convert both to string to be safe (ids like '1907311' preserve formatting).
df_daten['id'] = df_daten['id'].astype(str)
df_missing['id'] = df_missing['id'].astype(str)

# merge df_daten with df_missing to include dates with Umsatz = 0
df_umsatz_merged = df_daten.merge(df_missing, on=["Datum", "id", "Warengruppe", "Umsatz"], how="outer")


In [30]:
print(df_umsatz_merged)

            id      Datum  Warengruppe      Umsatz
0      1307011 2013-07-01            1  148.828353
1      1307012 2013-07-01            2  535.856285
2      1307013 2013-07-01            3  201.198426
3      1307014 2013-07-01            4   65.890169
4      1307015 2013-07-01            5  317.475875
...        ...        ...          ...         ...
11413  1907311 2019-07-31            1         NaN
11414  1907312 2019-07-31            2         NaN
11415  1907313 2019-07-31            3         NaN
11416  1907314 2019-07-31            4         NaN
11417  1907315 2019-07-31            5         NaN

[11418 rows x 4 columns]


In [31]:
#missing values in each column
print(df_umsatz_merged.isnull().sum())

id                0
Datum             0
Warengruppe       0
Umsatz         1897
dtype: int64


In [32]:
#save as csv
df_umsatz_merged.to_csv('/workspaces/bakery_prediction/0_DataPreparation/Processed/full_umsatzdaten.csv', index=False)
